<h1><center><font size="9">Preparing for Tableau Notebook</font></center></h1>

# <a id='0'>Table of Contents</a>

- <a href='#1'>Importing Packages</a>
- <a href='#2'>Importing Dataset and Adding Geographic Features</a>
- <a href='#3'>Adding Urbanization Level</a>
- <a href='#4'>Adding County Life Expectancy Data by Ethnicity</a>
- <a href='#5'>Joining with Main Dataset</a>
- <a href='#6'>Creating Percentile Rank by Year for Features</a>
- <a href='#7'>Difference in Life Expectancy Between Top and Bottom 1-Percent</a>
- <a href='#8'>Finalize Dataset for Upload</a>

# Importing Packages

In [1]:
import os

os.environ["USE_PYGEOS"] = "0"

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
import seaborn as sns
import numpy as np

pd.options.display.max_rows = None  # Display all rows of a DataFrame
pd.options.display.max_columns = None  # Display all columns of a DataFrame
pd.options.mode.chained_assignment = None

import warnings

warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler

import statsmodels as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Importing Dataset and Adding Geographic Features

In [69]:
# Load your dataset (life expectancy by county)
data = pd.read_csv(
    "data/chr_census_cleaned.csv", index_col=0, dtype={"geo_code": str, "year": int}
)

data["state_fips"] = data["geo_code"].apply(lambda x: str(x)[:2])

data["region"] = np.where(
    data["state_fips"].isin(["09", "23", "25", "33", "44", "50", "34", "36", "42"]),
    "Northeast",
    np.where(
        data["state_fips"].isin(
            ["17", "18", "26", "39", "55", "19", "20", "27", "29", "31", "38", "46"]
        ),
        "Midwest",
        np.where(
            data["state_fips"].isin(
                [
                    "10",
                    "12",
                    "13",
                    "24",
                    "37",
                    "45",
                    "51",
                    "11",
                    "54",
                    "01",
                    "21",
                    "28",
                    "47",
                    "05",
                    "22",
                    "40",
                    "48",
                ]
            ),
            "South",
            np.where(
                data["state_fips"].isin(
                    [
                        "04",
                        "08",
                        "16",
                        "30",
                        "32",
                        "35",
                        "49",
                        "56",
                        "02",
                        "06",
                        "15",
                        "41",
                        "53",
                    ]
                ),
                "West",
                np.nan,
            ),
        ),
    ),
)

data["division"] = np.where(
    data["state_fips"].isin(["09", "23", "25", "33", "44", "50"]),
    "New England",
    np.where(
        data["state_fips"].isin(["34", "36", "42"]),
        "Middle Atlantic",
        np.where(
            data["state_fips"].isin(["17", "18", "26", "39", "55"]),
            "East North Central",
            np.where(
                data["state_fips"].isin(["19", "20", "27", "29", "31", "38", "46"]),
                "West North Central",
                np.where(
                    data["state_fips"].isin(
                        ["10", "12", "13", "24", "37", "45", "51", "11", "54"]
                    ),
                    "South Atlantic",
                    np.where(
                        data["state_fips"].isin(["01", "21", "28", "47"]),
                        "East South Central",
                        np.where(
                            data["state_fips"].isin(["05", "22", "40", "48"]),
                            "West South Central",
                            np.where(
                                data["state_fips"].isin(
                                    ["04", "08", "16", "30", "32", "35", "49", "56"]
                                ),
                                "Mountain",
                                np.where(
                                    data["state_fips"].isin(
                                        ["02", "06", "15", "41", "53"]
                                    ),
                                    "Pacific",
                                    np.nan,
                                ),
                            ),
                        ),
                    ),
                ),
            ),
        ),
    ),
)

data.head()

,geo_code,year,geo_full_name,geo_name,state,state_code,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_asian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.281400,12.9412,0.138000,0.217477,0.000259,0.11896,0.000000,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.8,52.224775,0.004963,0.006547,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.1,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central
1,01003,2010,"Baldwin County, Alabama",Baldwin,Alabama,AL,0.245,0.230900,11.8769,0.193264,0.268221,0.000478,0.11214,0.096951,0.835134,0.177384,0.133441,68.766137,0.767301,101093.0,63.589498,4.214440,0.334602,30.659218,77.707111,-87.746067,0.0877,0.664884,50147.0,0.000788,21.87,23.6,70.741370,0.007428,0.006536,0.511340,0.068492,0.038955,0.273230,0.004,0.022074,0.098688,0.423983,110.576018,175791.0,8221.7,65.391299,0.116501,245.9,0.138206,11.396557,31.854171,0.065605,0.045870,171.623233,2.675232,0.485230,2.587933,01,South,East South Central
2,01005,2010,"Barbour County, Alabama",Barbour,Alabama,AL,0.364,0.226800,12.3622,0.341000,0.135124,0.000361,0.14666,0.246521,0.789877,0.127674,0.151423,74.681632,0.680449,12011.0,13.573725,5.835112,0.394364,31.870670,75.719796,-85.405456,0.1100,0.635756,33219.0,0.000050,20.08,30.4,40.744262,0.005806,0.001661,0.471028,0.037945,0.045958,0.261057,0.000,0.021339,0.039156,0.678638,31.302857,27699.0,10686.1,92.297198,0.158040,585.7,0.225625,8.381110,87.486157,0.095894,0.051303,64.005310,1.885737,0.367540,3.398067,01,South,East South Central
3,01007,2010,"Bibb County, Alabama",Bibb,Alabama,AL,0.317,0.249137,12.7501,0.242000,0.100252,0.000179,0.12166,0.000000,0.849911,0.122094,0.139013,69.830180,0.829190,8885.0,14.271285,4.332317,0.395793,33.015893,73.576596,-87.127148,0.0911,0.610994,41770.0,0.000073,10.39,41.3,42.105263,0.003612,0.001592,0.462096,0.057236,0.005927,0.259086,0.002,0.008245,0.046746,0.697744,36.316686,22610.0,13069.5,96.794256,0.066255,284.0,0.187878,10.242723,30.716724,0.091338,0.034706,127.580788,2.620188,0.340095,3.237233,01,South,East South Central
4,01009,2010,"Blount County, Alabama",Blount,Alabama,AL,0.315,0.234200,12.3118,0.185000,0.125272,0.000190,0.12005,0.005384,0.806364,0.088601,0.150667,73.733161,0.820044,23482.0,36.419060,4.310364,0.367124,33.977448,75.623747,-86.567246,0.0726,0.607473,45549.0,0.000046,14.31,28.6,19.987281,0.006218,0.004339,0.504692,0.046264,0.074896,0.259615,0.006,0.042746,0.066947,0.899569,87.925617,56692.0,8929.7,102.021063,0.068867,85.1,0.142587,8.432691,36.640361,0.075270,0.046817,93.782854,2.817210,0.274598,3.446433,01,South,East South Central


# Adding Urbanization level

**Metropolitan counties:**
1. **Large central metro**
   - Counties in MSAs of 1 million or more population that: 
      1. Contain the entire population of the largest principal city of the MSA, or 
      2. Have their entire population contained in the largest principal city of the MSA, or 
      3. Contain at least 250,000 inhabitants of any principal city of the MSA
2. **Large fringe metro:**
   - Counties in MSAs of 1 million or more population that did not qualify as large central metro counties
3. **Medium metro:**
   - Counties in MSAs of populations of 250,000–999,999
4. **Small metro:**
   - Counties in MSAs of populations less than 250,000

**Nonmetropolitan counties:**

5. **Micropolitan:**
    - Counties in micropolitan statistical areas
6. **Non-core:**
    - counties that did not qualify as Core-Based Statistical Areas, i.e. metropolitan or metropolitan statistical areas 
 

In [70]:
df_list = []
year_list = ["2013", "2015", "2017", "2018", "2019", "2020", "2023"]

cbsa_msa_2009 = pd.read_excel(
    "data/Urban Rural/cbsa_msa_county_2003_2023.xlsx",
    sheet_name="2009",
    dtype={"cbsa_code": str, "geo_code": str},
)
cbsa_msa_2009["year"] = 2010

df_list.append(cbsa_msa_2009)

cbsa_msa_2009.head()

,CBSA Title,cbsa_code,county_name,state_name,geo_code,county_status,cbsa_level,csa_status,cbsa_status,year
0,"Montgomery, AL",33860,Autauga County,Alabama,01001,Outlying,Metro,CSA,CBSA,2010
1,"Daphne-Fairhope-Foley, AL",19300,Baldwin County,Alabama,01003,Central,Micro,CSA,CBSA,2010
2,"Eufaula, AL-GA",21640,Barbour County,Alabama,01005,Central,Micro,non-CSA,CBSA,2010
3,"Birmingham-Hoover, AL",13820,Bibb County,Alabama,01007,Outlying,Metro,CSA,CBSA,2010
4,"Birmingham-Hoover, AL",13820,Blount County,Alabama,01009,Outlying,Metro,CSA,CBSA,2010


In [71]:
for year in year_list:
    cbsa_msa = pd.read_excel(
        "data/Urban Rural/cbsa_msa_county_2003_2023.xlsx",
        sheet_name=year,
        dtype={"cbsa_code": str, "state_fips": str, "county_fips": str},
    )
    cbsa_msa["geo_code"] = cbsa_msa["state_fips"] + cbsa_msa["county_fips"]
    cbsa_msa.drop(columns=["state_fips", "county_fips"], inplace=True)
    cbsa_msa["year"] = int(year)
    df_list.append(cbsa_msa)

In [72]:
cbsa_msa = pd.concat(df_list, axis=0)
cbsa_msa.head()

,CBSA Title,cbsa_code,county_name,state_name,geo_code,county_status,cbsa_level,csa_status,cbsa_status,year
0,"Montgomery, AL",33860,Autauga County,Alabama,01001,Outlying,Metro,CSA,CBSA,2010
1,"Daphne-Fairhope-Foley, AL",19300,Baldwin County,Alabama,01003,Central,Micro,CSA,CBSA,2010
2,"Eufaula, AL-GA",21640,Barbour County,Alabama,01005,Central,Micro,non-CSA,CBSA,2010
3,"Birmingham-Hoover, AL",13820,Bibb County,Alabama,01007,Outlying,Metro,CSA,CBSA,2010
4,"Birmingham-Hoover, AL",13820,Blount County,Alabama,01009,Outlying,Metro,CSA,CBSA,2010


In [73]:
pop_2020 = pd.read_csv(
    "data/Urban Rural/population_files/ACSDP5Y2020.DP05-Data.csv",
    header=1,
)
pop_2020 = pop_2020.iloc[:, :3]
pop_2020.columns = ["cbsa_code", "cbsa_title", "population"]
pop_2020["cbsa_title"] = pop_2020["cbsa_title"].apply(lambda x: str(x)[:-10])
pop_2020["cbsa_code"] = pop_2020["cbsa_code"].apply(lambda x: str(x)[-5:])
pop_2020["year"] = 2020
pop_2020.head()

,cbsa_code,cbsa_title,population,year
0,000US,Uni,326569308,2020
1,10100,"Aberdeen, SD",42864,2020
2,10140,"Aberdeen, WA",73769,2020
3,10180,"Abilene, TX",171354,2020
4,10220,"Ada, OK",38385,2020


In [74]:
year_list = ["2010", "2013", "2015", "2017", "2018", "2019", "2020", "2023"]

df_list = []

for year in year_list:
    if year == "2023":
        pop = pd.read_csv(
            f"data/Urban Rural/population_files/ACSDP1Y{year}.DP05-Data.csv",
            header=1,
        )
    else:
        pop = pd.read_csv(
            f"data/Urban Rural/population_files/ACSDP5Y{year}.DP05-Data.csv",
            header=1,
        )
    pop = pop.iloc[:, :3]
    pop.columns = ["cbsa_code", "cbsa_title", "population"]
    pop["year"] = int(year)
    pop["cbsa_code"] = pop["cbsa_code"].apply(lambda x: str(x)[-5:])
    pop = pop.pivot_table(
        index=["cbsa_code"], columns="year", values="population", aggfunc="sum"
    )
    df_list.append(pop)

In [75]:
pop_df = (
    df_list[0]
    .join(df_list[1], how="outer")
    .join(df_list[2], how="outer")
    .join(df_list[3], how="outer")
    .join(df_list[4], how="outer")
    .join(df_list[5], how="outer")
    .join(df_list[6], how="outer")
    .join(df_list[7], how="outer")
)
pop_df.head()

year,2010,2013,2015,2017,2018,2019,2020,2023
cbsa_code,,,,,,,,
000US,303965272.0,311536594.0,316515021.0,321004407.0,322903030.0,324697795.0,326569308.0,NaN
10020,57280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10100,40058.0,41033.0,42078.0,42608.0,42780.0,42824.0,42864.0,NaN
10140,72092.0,72092.0,71419.0,71454.0,71967.0,72779.0,73769.0,77290.0
10180,163092.0,166202.0,167945.0,169000.0,170009.0,170669.0,171354.0,180224.0


In [76]:
pop_pivot = pop_df.stack().reset_index()
pop_pivot.columns = ["cbsa_code", "year", "cbsa_population"]
pop_pivot.head()

,cbsa_code,year,cbsa_population
0,000US,2010,303965272.0
1,000US,2013,311536594.0
2,000US,2015,316515021.0
3,000US,2017,321004407.0
4,000US,2018,322903030.0


In [77]:
merged = cbsa_msa.merge(pop_pivot, on=["cbsa_code", "year"], how="left")
merged.head()

,CBSA Title,cbsa_code,county_name,state_name,geo_code,county_status,cbsa_level,csa_status,cbsa_status,year,cbsa_population
0,"Montgomery, AL",33860,Autauga County,Alabama,01001,Outlying,Metro,CSA,CBSA,2010,370554.0
1,"Daphne-Fairhope-Foley, AL",19300,Baldwin County,Alabama,01003,Central,Micro,CSA,CBSA,2010,175791.0
2,"Eufaula, AL-GA",21640,Barbour County,Alabama,01005,Central,Micro,non-CSA,CBSA,2010,30227.0
3,"Birmingham-Hoover, AL",13820,Bibb County,Alabama,01007,Outlying,Metro,CSA,CBSA,2010,1115485.0
4,"Birmingham-Hoover, AL",13820,Blount County,Alabama,01009,Outlying,Metro,CSA,CBSA,2010,1115485.0


In [78]:
data_head = data[["geo_code", "year"]]
data_head = data_head.loc[
    data_head["year"].isin([2010, 2013, 2015, 2017, 2018, 2019, 2020, 2023])
]
data_merge = data_head.merge(merged, on=["geo_code", "year"], how="left")
data_merge.head()

,geo_code,year,CBSA Title,cbsa_code,county_name,state_name,county_status,cbsa_level,csa_status,cbsa_status,cbsa_population
0,01001,2010,"Montgomery, AL",33860,Autauga County,Alabama,Outlying,Metro,CSA,CBSA,370554.0
1,01003,2010,"Daphne-Fairhope-Foley, AL",19300,Baldwin County,Alabama,Central,Micro,CSA,CBSA,175791.0
2,01005,2010,"Eufaula, AL-GA",21640,Barbour County,Alabama,Central,Micro,non-CSA,CBSA,30227.0
3,01007,2010,"Birmingham-Hoover, AL",13820,Bibb County,Alabama,Outlying,Metro,CSA,CBSA,1115485.0
4,01009,2010,"Birmingham-Hoover, AL",13820,Blount County,Alabama,Outlying,Metro,CSA,CBSA,1115485.0


In [79]:
data_merge.loc[data_merge.cbsa_status.isnull(), "cbsa_status"] = "Non-CBSA"
data_merge.loc[data_merge.cbsa_level.isnull(), "cbsa_level"] = "Non-Core"

In [80]:
for index, row in data_merge.iterrows():
    if (
        (row["cbsa_level"] == "Metro")
        & (row["county_status"] == "Central")
        & (row["cbsa_population"] >= 1000000)
    ):
        data_merge.loc[index, "cbsa_status"] = "Large Central Metro"
    elif (
        (row["cbsa_level"] == "Metro")
        & (row["county_status"] == "Outlying")
        & (row["cbsa_population"] >= 1000000)
    ):
        data_merge.loc[index, "cbsa_status"] = "Large Outlying Metro"
    elif (
        (row["cbsa_level"] == "Metro")
        & (row["cbsa_population"] < 1000000)
        & (row["cbsa_population"] >= 250000)
    ):
        data_merge.loc[index, "cbsa_status"] = "Medium Metro"
    elif (row["cbsa_level"] == "Metro") & (row["cbsa_population"] < 250000):
        data_merge.loc[index, "cbsa_status"] = "Small Metro"
    elif row["cbsa_level"] == "Micro":
        data_merge.loc[index, "cbsa_status"] = "Micropolitan"
    elif row["cbsa_level"] == "Non-Core":
        data_merge.loc[index, "cbsa_status"] = "Non-Core"
    else:
        data_merge.loc[index, "cbsa_status"] = np.nan

In [81]:
df = data.merge(
    data_merge[["geo_code", "year", "cbsa_status"]], on=["geo_code", "year"], how="left"
)
df.head()

,geo_code,year,geo_full_name,geo_name,state,state_code,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_asian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division,cbsa_status
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.281400,12.9412,0.138000,0.217477,0.000259,0.11896,0.000000,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.8,52.224775,0.004963,0.006547,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.1,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central,Medium Metro
1,01003,2010,"Baldwin County, Alabama",Baldwin,Alabama,AL,0.245,0.230900,11.8769,0.193264,0.268221,0.000478,0.11214,0.096951,0.835134,0.177384,0.133441,68.766137,0.767301,101093.0,63.589498,4.214440,0.334602,30.659218,77.707111,-87.746067,0.0877,0.664884,50147.0,0.000788,21.87,23.6,70.741370,0.007428,0.006536,0.511340,0.068492,0.038955,0.273230,0.004,0.022074,0.098688,0.423983,110.576018,175791.0,8221.7,65.391299,0.116501,245.9,0.138206,11.396557,31.854171,0.065605,0.045870,171.623233,2.675232,0.485230,2.587933,01,South,East South Central,Micropolitan
2,01005,2010,"Barbour County, Alabama",Barbour,Alabama,AL,0.364,0.226800,12.3622,0.341000,0.135124,0.000361,0.14666,0.246521,0.789877,0.127674,0.151423,74.681632,0.680449,12011.0,13.573725,5.835112,0.394364,31.870670,75.719796,-85.405456,0.1100,0.635756,33219.0,0.000050,20.08,30.4,40.744262,0.005806,0.001661,0.471028,0.037945,0.045958,0.261057,0.000,0.021339,0.039156,0.678638,31.302857,27699.0,10686.1,92.297198,0.158040,585.7,0.225625,8.381110,87.486157,0.095894,0.051303,64.005310,1.885737,0.367540,3.398067,01,South,East South Central,Micropolitan
3,01007,2010,"Bibb County, Alabama",Bibb,Alabama,AL,0.317,0.249137,12.7501,0.242000,0.100252,0.000179,0.12166,0.000000,0.849911,0.122094,0.139013,69.830180,0.829190,8885.0,14.271285,4.332317,0.395793,33.015893,73.576596,-87.127148,0.0911,0.610994,41770.0,0.000073,10.39,41.3,42.105263,0.003612,0.001592,0.462096,0.057236,0.005927,0.259086,0.002,0.008245,0.046746,0.697744,36.316686,22610.0,13069.5,96.794256,0.066255,284.0,0.187878,10.242723,30.716724,0.091338,0.034706,127.580788,2.620188,0.340095,3.237233,01,South,East South Central,Large Outlying Metro
4,01009,2010,"Blount County, Alabama",Blount,Alabama,AL,0.315,0.234200,12.3118,0.185000,0.125272,0.000190,0.12005,0.005384,0.806364,0.088601,0.150667,73.733161,0.820044,23482.0,36.419060,4.310364,0.367124,33.977448,75.623747,-86.567246,0.0726,0.607473,45549.0,0.000046,14.31,28.6,19.987281,0.006218,0.004339,0.504692,0.046264,0.074896,0.259615,0.006,0.042746,0.066947,0.899569,87.925617,56692.0,8929.7,102.021063,0.068867,85.1,0.142587,8.432691,36.640361,0.075270,0.046817,93.782854,2.817210,0.274598,3.446433,01,South,East South Central,Large Outlying Metro


In [82]:
df.sort_values(by=["geo_code", "year"], inplace=True)

df["cbsa_status"] = df.groupby("geo_code")["cbsa_status"].ffill()
df["cbsa_status"] = df.groupby("geo_code")["cbsa_status"].bfill()

In [83]:
df = df.reset_index(drop=True)

In [84]:
df.head()

,geo_code,year,geo_full_name,geo_name,state,state_code,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_asian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division,cbsa_status
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.2814,12.9412,0.138000,0.217477,0.000259,0.11896,0.0,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.800000,52.224775,0.004963,0.006547,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.10,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central,Medium Metro
1,01001,2011,"Autauga County, Alabama",Autauga,Alabama,AL,0.315,0.2740,12.7858,0.149000,0.216246,0.000246,0.11400,0.0,0.873219,0.141000,0.130358,65.685521,0.777478,21859.0,36.772874,3.899220,0.365611,32.536382,75.400943,-86.644490,0.0945,0.695122,51622.0,0.000020,19.53,28.300000,0.000437,0.005000,0.007137,0.514793,0.061602,0.024062,0.254795,0.002,0.010779,0.070508,0.448000,90.748703,53944.0,9967.40,78.853600,0.098860,307.8,0.170717,13.412318,42.572062,0.075488,0.028486,256.300000,2.499037,0.503662,3.186000,01,South,East South Central,Medium Metro
2,01001,2012,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2460,12.7350,0.159811,0.217078,0.000210,0.12300,0.0,0.877878,0.159000,0.129745,63.094541,0.776964,22077.0,37.139559,4.037353,0.365431,32.536382,75.618037,-86.644490,0.0970,0.638177,53049.0,0.000020,16.92,28.383137,0.000437,0.005000,0.008042,0.513867,0.061637,0.023997,0.259176,0.001,0.007834,0.074197,0.448000,91.835327,54590.0,9865.83,90.710834,0.107705,424.9,0.177385,13.412292,44.312471,0.085724,0.027875,290.509373,2.521405,0.505396,3.074000,01,South,East South Central,Medium Metro
3,01001,2013,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2350,13.3100,0.174717,0.209323,0.000256,0.12300,0.0,0.885189,0.165000,0.128714,65.520777,0.768173,22220.0,37.380124,4.186341,0.365516,32.532160,75.703890,-86.646469,0.0940,0.684659,48863.0,0.000037,18.36,23.917000,0.000366,0.004831,0.009562,0.512029,0.059615,0.024860,0.260628,0.001,0.005380,0.078205,0.420022,92.368608,54907.0,8376.42,70.950000,0.122515,362.8,0.162124,13.442152,35.749752,0.093913,0.029669,300.336683,2.487776,0.495594,3.210667,01,South,East South Central,Medium Metro
4,01001,2014,"Autauga County, Alabama",Autauga,Alabama,AL,0.305,0.2170,12.9200,0.186381,0.219413,0.000270,0.11800,0.0,0.892301,0.167000,0.131703,66.392790,0.750985,22431.0,37.735084,4.338252,0.366156,32.532160,76.094028,-86.646469,0.0930,0.665882,51441.0,0.000100,20.15,23.917000,0.000380,0.005091,0.009068,0.514401,0.059686,0.025229,0.264174,0.002,0.004387,0.081556,0.420022,92.753849,55136.0,8376.42,75.145591,0.123178,446.9,0.152088,13.360618,14.871795,0.085319,0.029689,302.909674,2.559708,0.443494,2.976000,01,South,East South Central,Medium Metro


In [85]:
numeric_data = df.select_dtypes(include=[np.number])

# Calculating the correlation matrix between life expectancy and all other numeric variables
correlation_matrix = numeric_data.corr()

# Selecting correlations with life expectancy and sorting them
life_expectancy_corr = correlation_matrix["life_expectancy"].sort_values(
    ascending=False
)

# Display the top correlations (both positive and negative) with life expectancy
top_positive_corr = life_expectancy_corr.head(15)
top_negative_corr = life_expectancy_corr.tail(15)

pd.DataFrame(top_positive_corr)

,life_expectancy
life_expectancy,1.000000
college_completion,0.573507
median_household_income,0.531112
food_access_score,0.448437
excessive_drinking,0.400704
latitude,0.346018
pct_asian,0.320067
pct_finance_insurance_and_real_estate,0.310775
pct_professional_management_and_scientific_services,0.297328
mammography_screening,0.279283


In [87]:
pd.DataFrame(top_negative_corr)

,life_expectancy
income_inequality,-0.263399
driving_alone_to_work,-0.314795
unemployment_rate,-0.331319
motor_vehicle_crash_deaths,-0.372814
sexually_transmitted_infections,-0.373562
single_parent_households,-0.439138
low_birthweight,-0.497623
adult_obesity,-0.564590
insufficient_sleep,-0.572529
adult_smoking,-0.580674


In [87]:
df.to_csv('data/chr_census_dataset_eda.csv')

# Adding County Life Expectancy Data by Ethnicity

In [18]:
df_list = []

year_list = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

for year in year_list:
    life_exp = pd.read_csv(
        f"data/IHME/IHME_USA_LE_COUNTY_RACE_ETHN_2000_2019_LT_{year}_BOTH_Y2022M06D16.CSV", dtype={"fips": str}
    )

    life_exp_race = life_exp.loc[(life_exp['age_group_id']==28) & (life_exp["race_id"]!=1)]
    life_exp_race = life_exp_race[['fips', 'race_name', 'year', 'val']]
    df_list.append(life_exp_race)



In [19]:
county_race_life_exp = pd.concat(df_list, axis=0)
county_race_life_exp = county_race_life_exp.reset_index(drop=True)
county_race_life_exp.columns = ['geo_code', 'col_name', 'year', 'val']
county_race_life_exp.head()

,geo_code,col_name,year,val
0,NaN,Latino,2010,81.853618
1,NaN,Black,2010,74.805665
2,NaN,White,2010,78.805920
3,NaN,AIAN,2010,73.490754
4,NaN,API,2010,84.903040


In [20]:
county_race_life_exp.loc[county_race_life_exp['col_name'] == 'Latino', 'col_name'] = 'life_exp_hispanic'
county_race_life_exp.loc[county_race_life_exp['col_name'] == 'AIAN', 'col_name'] = 'life_exp_aian'
county_race_life_exp.loc[county_race_life_exp['col_name'] == 'API', 'col_name'] = 'life_exp_aapi'
county_race_life_exp.loc[county_race_life_exp['col_name'] == 'Black', 'col_name'] = 'life_exp_black'
county_race_life_exp.loc[county_race_life_exp['col_name'] == 'White', 'col_name'] = 'life_exp_white'

In [21]:
county_race_life_exp = county_race_life_exp.dropna(subset=['geo_code'])
cnty_race = county_race_life_exp.loc[county_race_life_exp['geo_code'].str.len() >= 4]
for index, row in cnty_race.iterrows():
    if len(row['geo_code']) == 4:
        cnty_race.loc[index, 'geo_code'] = '0' + row['geo_code']
    else:
        pass

cnty_race.head()

,geo_code,col_name,year,val
10,01001,life_exp_hispanic,2010,89.250721
11,01001,life_exp_black,2010,73.046680
12,01001,life_exp_white,2010,75.962840
13,01001,life_exp_aian,2010,NaN
14,01001,life_exp_aapi,2010,NaN


In [22]:
cnty_race_2010_2019 = cnty_race.pivot_table(index=['geo_code', 'year'], columns=['col_name'], values='val').reset_index()
cnty_race_2010_2019.head()

col_name,geo_code,year,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
0,01001,2010,NaN,NaN,73.046680,89.250721,75.962840
1,01001,2011,NaN,NaN,73.105158,88.820510,75.690610
2,01001,2012,NaN,NaN,73.333216,88.773309,75.909067
3,01001,2013,NaN,NaN,73.400482,88.672297,75.992891
4,01001,2014,NaN,NaN,73.878674,88.952039,76.363647


In [23]:
year_list = [2020, 2021, 2022, 2023, 2024]

df_list = []
for year in year_list:
    chr = pd.read_csv(f"data/County Health Rankings/{year}/analytic_data{year}.csv", header=1, dtype={"fipscode": str})

    chr = chr[["fipscode", "year", "v147_race_aian", "v147_race_asian", "v147_race_black", "v147_race_hispanic", "v147_race_white"]]
    chr.columns = ["geo_code", "year", "life_exp_aian", "life_exp_aapi", "life_exp_black", "life_exp_hispanic", "life_exp_white"]
    df_list.append(chr)

df_2020_2024 = pd.concat(df_list, axis=0)
df_2020_2024.head()

,geo_code,year,life_exp_aian,life_exp_aapi,life_exp_black,life_exp_hispanic,life_exp_white
0,00000,2020,76.865497,87.711804,75.491441,83.733348,78.817348
1,01000,2020,106.663845,87.895586,73.584266,96.996818,75.726944
2,01001,2020,NaN,NaN,75.044708,NaN,76.985870
3,01003,2020,NaN,85.658204,75.728780,91.830607,78.366769
4,01005,2020,NaN,NaN,73.657375,NaN,76.930717


In [24]:
cnty_race_2020_2024 = df_2020_2024[~df_2020_2024["geo_code"].str.endswith("000")]
cnty_race_2020_2024.head()

,geo_code,year,life_exp_aian,life_exp_aapi,life_exp_black,life_exp_hispanic,life_exp_white
2,01001,2020,NaN,NaN,75.044708,NaN,76.985870
3,01003,2020,NaN,85.658204,75.728780,91.830607,78.366769
4,01005,2020,NaN,NaN,73.657375,NaN,76.930717
5,01007,2020,NaN,NaN,72.887469,NaN,73.273066
6,01009,2020,NaN,NaN,NaN,NaN,73.647170


In [25]:
cnty_race = pd.concat([cnty_race_2010_2019, cnty_race_2020_2024], axis=0)
cnty_race.head()

,geo_code,year,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
0,01001,2010,NaN,NaN,73.046680,89.250721,75.962840
1,01001,2011,NaN,NaN,73.105158,88.820510,75.690610
2,01001,2012,NaN,NaN,73.333216,88.773309,75.909067
3,01001,2013,NaN,NaN,73.400482,88.672297,75.992891
4,01001,2014,NaN,NaN,73.878674,88.952039,76.363647


## Addresing FIPS Changes since 2010

In [26]:
counties_to_combine = ["46113", "46102"]
subset_sd = cnty_race.loc[(cnty_race["geo_code"].isin(counties_to_combine))]
subset_sd

,geo_code,year,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
23950,46102,2010,NaN,65.557177,NaN,NaN,NaN
23951,46102,2011,NaN,65.383844,NaN,NaN,NaN
23952,46102,2012,NaN,65.331333,NaN,NaN,NaN
23953,46102,2013,NaN,65.058039,NaN,NaN,NaN
23954,46102,2014,NaN,64.497195,NaN,NaN,NaN
23955,46102,2015,NaN,64.330736,NaN,NaN,NaN
23956,46102,2016,NaN,63.593985,NaN,NaN,NaN
23957,46102,2017,NaN,63.736961,NaN,NaN,NaN
23958,46102,2018,NaN,63.502431,NaN,NaN,NaN
23959,46102,2019,NaN,63.292512,NaN,NaN,NaN


In [27]:
counties_to_combine = ["51515", "51019"]

subset_va = cnty_race.loc[(cnty_race["geo_code"].isin(counties_to_combine))]
subset_va

,geo_code,year,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
28030,51019,2010,NaN,NaN,76.179838,87.865747,78.869357
28031,51019,2011,NaN,NaN,76.280548,88.036167,78.785518
28032,51019,2012,NaN,NaN,76.257792,87.922829,78.921462
28033,51019,2013,NaN,NaN,76.208013,87.871679,78.942793
28034,51019,2014,NaN,NaN,76.562312,88.123173,79.213304
28035,51019,2015,NaN,NaN,76.296368,87.978705,78.967435
28036,51019,2016,NaN,NaN,76.199932,88.036021,78.920966
28037,51019,2017,NaN,NaN,76.316460,88.111672,78.985550
28038,51019,2018,NaN,NaN,76.206907,88.071542,79.017957
28039,51019,2019,NaN,NaN,76.365358,88.142037,79.255216


In [28]:
counties_to_combine = ["02261", "02063", "02066"]
subset_ak = cnty_race.loc[(cnty_race["geo_code"].isin(counties_to_combine))]

subset_ak

,geo_code,year,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
720,02063,2010,NaN,68.962693,NaN,NaN,78.807561
721,02063,2011,NaN,69.387653,NaN,NaN,79.178591
722,02063,2012,NaN,69.487220,NaN,NaN,79.484697
723,02063,2013,NaN,69.483163,NaN,NaN,79.838231
724,02063,2014,NaN,69.866899,NaN,NaN,80.518255
725,02063,2015,NaN,69.443244,NaN,NaN,80.750343
726,02063,2016,NaN,69.050056,NaN,NaN,80.735795
727,02063,2017,NaN,70.149091,NaN,NaN,81.811389
728,02063,2018,NaN,69.940235,NaN,NaN,81.841910
729,02063,2019,NaN,70.179961,NaN,NaN,82.455996


In [29]:
# Assume you want to combine data for counties with FIPS codes '12345' and '67890'
counties_to_combine = ["02270", "02158"]
subset = cnty_race.loc[(cnty_race["geo_code"].isin(counties_to_combine))]

subset

,geo_code,year,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
840,02158,2010,NaN,68.683020,NaN,NaN,NaN
841,02158,2011,NaN,68.674964,NaN,NaN,NaN
842,02158,2012,NaN,68.828815,NaN,NaN,NaN
843,02158,2013,NaN,68.653270,NaN,NaN,NaN
844,02158,2014,NaN,68.898712,NaN,NaN,NaN
845,02158,2015,NaN,68.208812,NaN,NaN,NaN
846,02158,2016,NaN,67.464842,NaN,NaN,NaN
847,02158,2017,NaN,67.853820,NaN,NaN,NaN
848,02158,2018,NaN,68.623782,NaN,NaN,NaN
849,02158,2019,NaN,68.376682,NaN,NaN,NaN


In [30]:
counties_to_combine = ["02195", "02280"]
subset_ak = cnty_race.loc[
    (cnty_race["geo_code"].isin(counties_to_combine))
]
subset_ak

,geo_code,year,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
900,02195,2010,86.788646,71.968878,NaN,NaN,78.738693
901,02195,2011,87.518178,72.584882,NaN,NaN,79.276579
902,02195,2012,87.735193,72.937887,NaN,NaN,79.713423
903,02195,2013,87.457337,72.882533,NaN,NaN,79.718830
904,02195,2014,88.236134,72.745543,NaN,NaN,80.171918
905,02195,2015,88.374701,72.407374,NaN,NaN,80.323135
906,02195,2016,88.939484,72.261453,NaN,NaN,80.747045
907,02195,2017,89.488990,72.687672,NaN,NaN,81.563396
908,02195,2018,89.537972,72.695266,NaN,NaN,81.888167
909,02195,2019,89.808835,72.620334,NaN,NaN,82.566855


# Joining with Main Dataset

In [62]:
data = pd.read_csv(
    "data/chr_census_dataset_eda.csv", index_col=0, dtype={"geo_code": str, "year": int, "state_fips": str}
)

data.head()

,geo_code,year,geo_full_name,geo_name,state,state_code,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_asian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division,cbsa_status
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.2814,12.9412,0.138000,0.217477,0.000259,0.11896,0.0,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.800000,52.224775,0.004963,0.006547,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.10,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central,Medium Metro
1,01001,2011,"Autauga County, Alabama",Autauga,Alabama,AL,0.315,0.2740,12.7858,0.149000,0.216246,0.000246,0.11400,0.0,0.873219,0.141000,0.130358,65.685521,0.777478,21859.0,36.772874,3.899220,0.365611,32.536382,75.400943,-86.644490,0.0945,0.695122,51622.0,0.000020,19.53,28.300000,0.000437,0.005000,0.007137,0.514793,0.061602,0.024062,0.254795,0.002,0.010779,0.070508,0.448000,90.748703,53944.0,9967.40,78.853600,0.098860,307.8,0.170717,13.412318,42.572062,0.075488,0.028486,256.300000,2.499037,0.503662,3.186000,01,South,East South Central,Medium Metro
2,01001,2012,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2460,12.7350,0.159811,0.217078,0.000210,0.12300,0.0,0.877878,0.159000,0.129745,63.094541,0.776964,22077.0,37.139559,4.037353,0.365431,32.536382,75.618037,-86.644490,0.0970,0.638177,53049.0,0.000020,16.92,28.383137,0.000437,0.005000,0.008042,0.513867,0.061637,0.023997,0.259176,0.001,0.007834,0.074197,0.448000,91.835327,54590.0,9865.83,90.710834,0.107705,424.9,0.177385,13.412292,44.312471,0.085724,0.027875,290.509373,2.521405,0.505396,3.074000,01,South,East South Central,Medium Metro
3,01001,2013,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2350,13.3100,0.174717,0.209323,0.000256,0.12300,0.0,0.885189,0.165000,0.128714,65.520777,0.768173,22220.0,37.380124,4.186341,0.365516,32.532160,75.703890,-86.646469,0.0940,0.684659,48863.0,0.000037,18.36,23.917000,0.000366,0.004831,0.009562,0.512029,0.059615,0.024860,0.260628,0.001,0.005380,0.078205,0.420022,92.368608,54907.0,8376.42,70.950000,0.122515,362.8,0.162124,13.442152,35.749752,0.093913,0.029669,300.336683,2.487776,0.495594,3.210667,01,South,East South Central,Medium Metro
4,01001,2014,"Autauga County, Alabama",Autauga,Alabama,AL,0.305,0.2170,12.9200,0.186381,0.219413,0.000270,0.11800,0.0,0.892301,0.167000,0.131703,66.392790,0.750985,22431.0,37.735084,4.338252,0.366156,32.532160,76.094028,-86.646469,0.0930,0.665882,51441.0,0.000100,20.15,23.917000,0.000380,0.005091,0.009068,0.514401,0.059686,0.025229,0.264174,0.002,0.004387,0.081556,0.420022,92.753849,55136.0,8376.42,75.145591,0.123178,446.9,0.152088,13.360618,14.871795,0.085319,0.029689,302.909674,2.559708,0.443494,2.976000,01,South,East South Central,Medium Metro


In [63]:
columns = ['geo_code', 
           'year',
           'pct_age_18_44',
            'pct_age_45_64',
            'pct_65_and_older',
            'pct_agriculture_hunting_and_mining',
            'pct_arts_entertainment_and_accommodation_services',
            'pct_construction',
            'pct_educational_and_health_services',
            'pct_information',
            'pct_manufacturing',
            'pct_public_administration',
            'pct_retail_trade',
            'pct_transportation_and_utilities',
            'pct_wholesale_trade',
            # 'pct_mgmt_bus_sci_art_occupations',
            # 'pct_nat_res_constr_maint_occupations',
            # 'pct_prod_transp_mat_moving_occupations',
            # 'pct_sales_and_office_occupations',
            # 'pct_service_occupations',
            # 'pct_moved_from_diff_county_diff_state',
            # 'pct_moved_from_diff_county_same_state',
            # 'pct_moved_within_same_county',
            # 'pct_households_with_high_housing_costs',
            # 'pct_households_with_lack_of_kitchen_or_plumbing_facilities',
            # 'pct_households_with_overcrowding',
            # 'pct_indigenous',
            'pct_nhpi',
            'pct_non_hispanic_black',
            'pct_non_hispanic_white']



In [64]:
data_full = pd.read_csv('data/chr_census_dataset.csv', index_col=0, dtype={"geo_code": str, "year": int})
data_full.head()

,geo_code,year,geo_full_name,geo_name,state,state_code,access_to_exercise_opportunities,adult_obesity,adult_smoking,air_pollution_particulate_matter,alcohol_impaired_driving_deaths,children_eligible_for_free_lunch,children_in_poverty,children_in_single_parent_households,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,food_environment_index,food_insecurity,frequent_mental_distress,frequent_physical_distress,gender_pay_gap,gini_index,high_school_completion,high_school_graduation,hiv_prevalence,homeownership,housing_units,hu_density,income_inequality,injury_deaths,insufficient_sleep,latitude,life_expectancy,limited_access_to_healthy_foods,long_commute_driving_alone,longitude,low_birthweight,mammography_screening,median_age,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,other_pcp,pcp,pct_65_and_older,pct_age_18_44,pct_age_45_64,pct_agriculture_hunting_and_mining,pct_aian,pct_arts_entertainment_and_accommodation_services,pct_asian,pct_construction,pct_educational_and_health_services,pct_female,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_finance_insurance_and_real_estate,pct_hispanic,pct_households_with_high_housing_costs,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,pct_indigenous,pct_information,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_manufacturing,pct_mgmt_bus_sci_art_occupations,pct_moved_from_abroad,pct_moved_from_diff_county_diff_state,pct_moved_from_diff_county_same_state,pct_moved_within_same_county,pct_nat_res_constr_maint_occupations,pct_nhpi,pct_non_hispanic_black,pct_non_hispanic_white,pct_not_proficient_in_english,pct_other_services,pct_prod_transp_mat_moving_occupations,pct_professional_management_and_scientific_services,pct_public_administration,pct_retail_trade,pct_rural,pct_sales_and_office_occupations,pct_service_occupations,pct_transportation_and_utilities,pct_under_18,pct_wholesale_trade,physical_inactivity,poor_mental_health_days,poor_or_fair_health,poor_physical_health_days,pop_density,population,poverty,premature_age_adjusted_mortality,premature_death,preventable_hospital_stays,ratio_of_pop_to_dentists,ratio_of_pop_to_mental_health,ratio_of_pop_to_pcp,ratio_of_pop_to_pcp_other_than_physicians,severe_housing_cost_burden,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,some_college,teen_births,unemployment_rate,uninsured,uninsured_adults,uninsured_children,violent_crime
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.659871,0.300,0.281400,12.9412,0.282668,0.161946,0.138000,0.295363,0.217477,0.000259,0.11896,0.000000,0.868566,0.160546,7.259172,0.13667,0.128822,0.124827,66.048439,0.405,0.853323,0.715190,184.260610,0.775180,21530.0,36.219406,3.930128,73.135700,0.367626,32.536382,75.607833,0.102426,31.206695,-86.644490,0.0883,0.668719,36.2,53255.0,0.000063,16.51,28.8,0.000201,52.224775,0.114006,0.360926,0.248048,0.009449,0.004963,0.086767,0.006547,0.077400,0.174315,0.515003,0.264694,0.361352,0.246320,0.127635,0.065322,0.023196,0.144260,0.009737,0.002029,0.002897,0.011257,0.290109,0.360473,0.249884,0.099535,0.141284,0.293291,0.002,0.030,0.037,0.068,0.115690,0.000000,0.178215,0.775524,0.009389,0.045520,0.131096,0.069964,0.116676,0.126659,0.422819,0.290703,0.169221,0.044246,0.277020,0.031141,0.29600,4.14,0.2586,5.48,89.421390,53155.0,0.188822,440.531154,9778.1,91.803383,3922.127612,32069.013341,1914.800000,5211.487030,0.144260,0.096612,370.0,0.179785,13.448252,0.501062,47.513321,0.062076,0.102987,0.140,0.028895,256.487012
1,01003,2010,"Baldwin County, Alabama",Baldwin,Alabama,AL,0.719029,0.245,0.230900,11.8769,0.311694,0.204360,0.193264,0.285221,0.268221,0.000478,0.11214,0.096951,0.835134,0.177384,7.839907,0.13262,0.133441,0.120058,68.766137,0.439,0.875757,0.670437,176.099286,0.767301,101093.0,63.589498,4.214440,73.584630,

In [65]:
data_merge = data.merge(data_full[columns], on=['geo_code', 'year'], how='left')
data_merge.head()

,geo_code,year,geo_full_name,geo_name,state,state_code,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_asian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division,cbsa_status,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_agriculture_hunting_and_mining,pct_arts_entertainment_and_accommodation_services,pct_construction,pct_educational_and_health_services,pct_information,pct_manufacturing,pct_public_administration,pct_retail_trade,pct_transportation_and_utilities,pct_wholesale_trade,pct_nhpi,pct_non_hispanic_black,pct_non_hispanic_white
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.2814,12.9412,0.138000,0.217477,0.000259,0.11896,0.0,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.800000,52.224775,0.004963,0.006547,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.10,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central,Medium Metro,0.360926,0.248048,0.114006,0.009449,0.086767,0.077400,0.174315,0.011257,0.141284,0.116676,0.126659,0.044246,0.031141,0.0,0.178215,0.775524
1,01001,2011,"Autauga County, Alabama",Autauga,Alabama,AL,0.315,0.2740,12.7858,0.149000,0.216246,0.000246,0.11400,0.0,0.873219,0.141000,0.130358,65.685521,0.777478,21859.0,36.772874,3.899220,0.365611,32.536382,75.400943,-86.644490,0.0945,0.695122,51622.0,0.000020,19.53,28.300000,0.000437,0.005000,0.007137,0.514793,0.061602,0.024062,0.254795,0.002,0.010779,0.070508,0.448000,90.748703,53944.0,9967.40,78.853600,0.098860,307.8,0.170717,13.412318,42.572062,0.075488,0.028486,256.300000,2.499037,0.503662,3.186000,01,South,East South Central,Medium Metro,0.358613,0.252187,0.116547,0.012469,0.081611,0.069804,0.183148,0.009942,0.139650,0.112722,0.124984,0.052985,0.036290,0.0,0.179668,0.772153
2,01001,2012,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2460,12.7350,0.159811,0.217078,0.000210,0.12300,0.0,0.877878,0.159000,0.129745,63.094541,0.776964,22077.0,37.139559,4.037353,0.365431,32.536382,75.618037,-86.644490,0.0970,0.638177,53049.0,0.000020,16.92,28.383137,0.000437,0.005000,0.008042,0.513867,0.061637,0.023997,0.259176,0.001,0.007834,0.074197,0.448000,91.835327,54590.0,9865.83,90.710834,0.107705,424.9,0.177385,13.412292,44.312471,0.085724,0.027875,290.509373,2.521405,0.505396,3.074000,01,South,East South Central,Medium Metro,0.353838,0.255450,0.122587,0.014342,0.078010,0.056456,0.187979,0.011233,0.135876,0.114031,0.134964,0.056870,0.029720,0.0,0.179447,0.769042
3,01001,2013,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2350,13.3100,0.174717,0.209323,0.000256,0.12300,0.0,0.885189,0.165000,0.128714,65.520777,0.768173,22220.0,37.380124,4.186341,0.365516,32.532160,75.703890,-86.646469,0.0940,0.684659,48863.0,0.000037,18.36,23.917000,0.000366,0.004831,0.009562,0.512029,0.059615,0.024860,0.260628,0.001,0.005380,0.078205,0.420022,92.368608,54907.0,8376.42,70.950000,0.122515,362.8,0.162124,13.442152,35.749752,0.093913,0.029669,300.3366

In [66]:
df_merge = data_merge.merge(cnty_race, on=['geo_code', 'year'], how='left')
df_merge.head()

,geo_code,year,geo_full_name,geo_name,state,state_code,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_asian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division,cbsa_status,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_agriculture_hunting_and_mining,pct_arts_entertainment_and_accommodation_services,pct_construction,pct_educational_and_health_services,pct_information,pct_manufacturing,pct_public_administration,pct_retail_trade,pct_transportation_and_utilities,pct_wholesale_trade,pct_nhpi,pct_non_hispanic_black,pct_non_hispanic_white,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.2814,12.9412,0.138000,0.217477,0.000259,0.11896,0.0,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.800000,52.224775,0.004963,0.006547,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.10,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central,Medium Metro,0.360926,0.248048,0.114006,0.009449,0.086767,0.077400,0.174315,0.011257,0.141284,0.116676,0.126659,0.044246,0.031141,0.0,0.178215,0.775524,NaN,NaN,73.046680,89.250721,75.962840
1,01001,2011,"Autauga County, Alabama",Autauga,Alabama,AL,0.315,0.2740,12.7858,0.149000,0.216246,0.000246,0.11400,0.0,0.873219,0.141000,0.130358,65.685521,0.777478,21859.0,36.772874,3.899220,0.365611,32.536382,75.400943,-86.644490,0.0945,0.695122,51622.0,0.000020,19.53,28.300000,0.000437,0.005000,0.007137,0.514793,0.061602,0.024062,0.254795,0.002,0.010779,0.070508,0.448000,90.748703,53944.0,9967.40,78.853600,0.098860,307.8,0.170717,13.412318,42.572062,0.075488,0.028486,256.300000,2.499037,0.503662,3.186000,01,South,East South Central,Medium Metro,0.358613,0.252187,0.116547,0.012469,0.081611,0.069804,0.183148,0.009942,0.139650,0.112722,0.124984,0.052985,0.036290,0.0,0.179668,0.772153,NaN,NaN,73.105158,88.820510,75.690610
2,01001,2012,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2460,12.7350,0.159811,0.217078,0.000210,0.12300,0.0,0.877878,0.159000,0.129745,63.094541,0.776964,22077.0,37.139559,4.037353,0.365431,32.536382,75.618037,-86.644490,0.0970,0.638177,53049.0,0.000020,16.92,28.383137,0.000437,0.005000,0.008042,0.513867,0.061637,0.023997,0.259176,0.001,0.007834,0.074197,0.448000,91.835327,54590.0,9865.83,90.710834,0.107705,424.9,0.177385,13.412292,44.312471,0.085724,0.027875,290.509373,2.521405,0.505396,3.074000,01,South,East South Central,Medium Metro,0.353838,0.255450,0.122587,0.014342,0.078010,0.056456,0.187979,0.011233,0.135876,0.114031,0.134964,0.056870,0.029720,0.0,0.179447,0.769042,NaN,NaN,73.333216,88.773309,75.909067
3,01001,2013,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2350,13.3100,0.174717,0.209323,0.000256,0.12300,0.0,0.885189,0.165000,0.128714,65.520777,0.768173,22220.0,37.380124,4.186341,0.365516,32.532160,75.703890,-86.646469,0.0940,0.684659,48863.0,0.000037,18.36,23.917000,0.000366,0.0

In [67]:
df = df_merge.copy()
df.head()

,geo_code,year,geo_full_name,geo_name,state,state_code,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_asian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division,cbsa_status,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_agriculture_hunting_and_mining,pct_arts_entertainment_and_accommodation_services,pct_construction,pct_educational_and_health_services,pct_information,pct_manufacturing,pct_public_administration,pct_retail_trade,pct_transportation_and_utilities,pct_wholesale_trade,pct_nhpi,pct_non_hispanic_black,pct_non_hispanic_white,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.2814,12.9412,0.138000,0.217477,0.000259,0.11896,0.0,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.800000,52.224775,0.004963,0.006547,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.10,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central,Medium Metro,0.360926,0.248048,0.114006,0.009449,0.086767,0.077400,0.174315,0.011257,0.141284,0.116676,0.126659,0.044246,0.031141,0.0,0.178215,0.775524,NaN,NaN,73.046680,89.250721,75.962840
1,01001,2011,"Autauga County, Alabama",Autauga,Alabama,AL,0.315,0.2740,12.7858,0.149000,0.216246,0.000246,0.11400,0.0,0.873219,0.141000,0.130358,65.685521,0.777478,21859.0,36.772874,3.899220,0.365611,32.536382,75.400943,-86.644490,0.0945,0.695122,51622.0,0.000020,19.53,28.300000,0.000437,0.005000,0.007137,0.514793,0.061602,0.024062,0.254795,0.002,0.010779,0.070508,0.448000,90.748703,53944.0,9967.40,78.853600,0.098860,307.8,0.170717,13.412318,42.572062,0.075488,0.028486,256.300000,2.499037,0.503662,3.186000,01,South,East South Central,Medium Metro,0.358613,0.252187,0.116547,0.012469,0.081611,0.069804,0.183148,0.009942,0.139650,0.112722,0.124984,0.052985,0.036290,0.0,0.179668,0.772153,NaN,NaN,73.105158,88.820510,75.690610
2,01001,2012,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2460,12.7350,0.159811,0.217078,0.000210,0.12300,0.0,0.877878,0.159000,0.129745,63.094541,0.776964,22077.0,37.139559,4.037353,0.365431,32.536382,75.618037,-86.644490,0.0970,0.638177,53049.0,0.000020,16.92,28.383137,0.000437,0.005000,0.008042,0.513867,0.061637,0.023997,0.259176,0.001,0.007834,0.074197,0.448000,91.835327,54590.0,9865.83,90.710834,0.107705,424.9,0.177385,13.412292,44.312471,0.085724,0.027875,290.509373,2.521405,0.505396,3.074000,01,South,East South Central,Medium Metro,0.353838,0.255450,0.122587,0.014342,0.078010,0.056456,0.187979,0.011233,0.135876,0.114031,0.134964,0.056870,0.029720,0.0,0.179447,0.769042,NaN,NaN,73.333216,88.773309,75.909067
3,01001,2013,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2350,13.3100,0.174717,0.209323,0.000256,0.12300,0.0,0.885189,0.165000,0.128714,65.520777,0.768173,22220.0,37.380124,4.186341,0.365516,32.532160,75.703890,-86.646469,0.0940,0.684659,48863.0,0.000037,18.36,23.917000,0.000366,0.0

In [37]:
df_merge.isna().sum()

geo_code                                                   0
year                                                       0
geo_full_name                                              0
geo_name                                                   0
state                                                      0
state_code                                                 0
adult_obesity                                              0
adult_smoking                                              0
air_pollution_particulate_matter                           0
children_in_poverty                                        0
college_completion                                         0
dentists                                                   0
diabetes_prevalence                                        0
drinking_water_violations                                  0
driving_alone_to_work                                      0
excessive_drinking                                         0
frequent_mental_distress

,life_expectancy
life_expectancy,1.000000
life_exp_white,0.904855
life_exp_black,0.736811
college_completion,0.573507
median_household_income,0.531112
food_access_score,0.448437
excessive_drinking,0.400704
life_exp_aian,0.380235
latitude,0.346018
pct_asian,0.320067


# Creating Percetile Ranks for Features

In [38]:
df['pct_aapi'] = df['pct_asian'] + df['pct_nhpi']
df.drop(columns=['pct_asian', 'pct_nhpi'], inplace=True)

In [39]:
df["percentiles_black_by_year"] = df.groupby("year")["pct_non_hispanic_black"].transform(
    lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)

df["percentiles_white_by_year"] = df.groupby("year")["pct_non_hispanic_white"].transform(
    lambda x: pd.qcut(x, 100, labels=False)
)
df["percentiles_hispanic_by_year"] = df.groupby("year")["pct_hispanic"].transform(
    lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)
df["percentiles_aapi_by_year"] = df.groupby("year")["pct_aapi"].transform(
    lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)
df["percentiles_aian_by_year"] = df.groupby("year")["pct_aian"].transform(
    lambda x: pd.qcut(
        x,
        100,
        labels=False,
        duplicates="drop",
    )
)
df["percentiles_rural_by_year"] = df.groupby("year")["pct_rural"].transform(
    lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)
df["percentiles_income_by_year"] = df.groupby("year")["median_household_income"].transform(
    lambda x: pd.qcut(x, 100, labels=False)
)
df["percentiles_college_by_year"] = df.groupby("year")["college_completion"].transform(
    lambda x: pd.qcut(x, 100, labels=False)
)
df["percentiles_unemployment_by_year"] = df.groupby("year")["unemployment_rate"].transform(
    lambda x: pd.qcut(x, 100, labels=False)
)
df["percentiles_not_english_proficient_by_year"] = df.groupby("year")[
    "pct_not_proficient_in_english"
].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates="drop"))

df["percentiles_obesity_by_year"] = df.groupby("year")["adult_obesity"].transform(
    lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)
df["percentiles_diabetes_by_year"] = df.groupby("year")[
    "diabetes_prevalence"
].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)
df["percentiles_smoking_by_year"] = df.groupby("year")["adult_smoking"].transform(
    lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)
df["percentiles_food_access_by_year"] = df.groupby("year")["food_access_score"].transform(
    lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)
df["percentiles_exercise_access_by_year"] = df.groupby("year")[
    "exercise_access_score"
].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates="drop")
)
df["percentiles_finance_by_year"] = df.groupby("year")[
    "pct_finance_insurance_and_real_estate"
].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates="drop"))
df["percentiles_professional_by_year"] = df.groupby("year")[
    "pct_professional_management_and_scientific_services"
].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates="drop"))
df["percentiles_single_parent_households_by_year"] = df.groupby("year")[
    "single_parent_households"
].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates="drop"))

df["percentiles_pop_density_by_year"] = df.groupby("year")[
    "pop_density"
].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates="drop"))

df["percentiles_hu_density_by_year"] = df.groupby("year")[
    "hu_density"
].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates="drop"))

df['percentiles_crime_by_year'] = df.groupby('year')['violent_crime'].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates='drop'))


df['percentiles_vehicle_crashes_by_year'] = df.groupby('year')['motor_vehicle_crash_deaths'].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates='drop'))

df['percentiles_housing_problems_by_year'] = df.groupby('year')['severe_housing_problems'].transform(lambda x: pd.qcut(x, 100, labels=False, duplicates='drop'))


# Difference in Life Expectancy Between Top and Bottom 1-Percent

In [40]:
pd.DataFrame(df.groupby("percentiles_single_parent_households_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_single_parent_households_by_year,
0,79.821514
99,72.161472


In [41]:
pd.DataFrame(df.groupby("percentiles_food_access_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_food_access_by_year,
0,73.685816
99,80.544281


In [42]:
pd.DataFrame(df.groupby("percentiles_finance_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_finance_by_year,
0,77.880745
99,80.599595


In [43]:
pd.DataFrame(df.groupby("percentiles_professional_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_professional_by_year,
0,77.734514
99,81.693994


In [44]:
pd.DataFrame(df.groupby("percentiles_obesity_by_year")["life_expectancy"].mean()).iloc[[0, -1]]

,life_expectancy
percentiles_obesity_by_year,
0,83.182048
99,73.932858


In [45]:
pd.DataFrame(df.groupby("percentiles_diabetes_by_year")["life_expectancy"].mean()).iloc[[0, -1]]

,life_expectancy
percentiles_diabetes_by_year,
0,82.294734
99,72.845071


In [46]:
pd.DataFrame(df.groupby("percentiles_smoking_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_smoking_by_year,
0,81.163888
99,72.053076


In [47]:
df.groupby(["cbsa_status"])["life_expectancy"].mean()

cbsa_status
Large Central Metro     79.283689
Large Outlying Metro    77.378252
Medium Metro            77.540237
Micropolitan            76.946827
Non-Core                76.845501
Small Metro             77.543179
Name: life_expectancy, dtype: float64

In [48]:
pd.DataFrame(df.groupby("percentiles_unemployment_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_unemployment_by_year,
0,79.463684
99,72.750254


In [49]:
pd.DataFrame(df.groupby("percentiles_college_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_college_by_year,
0,74.358504
99,82.835909


In [50]:
pd.DataFrame(df.groupby("percentiles_black_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_black_by_year,
0,78.918908
99,70.087400


In [51]:
pd.DataFrame(df.groupby("percentiles_obesity_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_obesity_by_year,
0,83.182048
99,73.932858


In [52]:
pd.DataFrame(df.groupby("percentiles_hispanic_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_hispanic_by_year,
0,76.184943
99,78.026502


In [53]:
pd.DataFrame(df.groupby("percentiles_aapi_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_aapi_by_year,
0,77.067645
99,82.103692


In [54]:
pd.DataFrame(df.groupby("percentiles_income_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_income_by_year,
0,72.005483
99,82.228036


In [55]:
pd.DataFrame(df.groupby("percentiles_rural_by_year")["life_expectancy"].mean()).iloc[
    [0, -1]
]

,life_expectancy
percentiles_rural_by_year,
0,78.691677
79,78.381038


In [56]:
df.groupby('region')['life_expectancy'].mean()

region
Midwest      78.070585
Northeast    79.036178
South        75.847745
West         78.925922
Name: life_expectancy, dtype: float64

In [57]:
df.groupby('division')['life_expectancy'].mean()

division
East North Central    77.643638
East South Central    74.368324
Middle Atlantic       78.839183
Mountain              78.704828
New England           79.477213
Pacific               79.297942
South Atlantic        76.588062
West North Central    78.372489
West South Central    76.067326
Name: life_expectancy, dtype: float64

# Finalizing Dataset for Upload

In [58]:
df.rename(columns={"geo_name": "county_name", "state_code": "state_abbr", "geo_full_name": "county_full_name", "state": "state_name"}, inplace=True)

In [59]:
df.head()

,geo_code,year,county_full_name,county_name,state_name,state_abbr,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division,cbsa_status,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_agriculture_hunting_and_mining,pct_arts_entertainment_and_accommodation_services,pct_construction,pct_educational_and_health_services,pct_information,pct_manufacturing,pct_public_administration,pct_retail_trade,pct_transportation_and_utilities,pct_wholesale_trade,pct_non_hispanic_black,pct_non_hispanic_white,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white,pct_aapi,percentiles_black_by_year,percentiles_white_by_year,percentiles_hispanic_by_year,percentiles_aapi_by_year,percentiles_aian_by_year,percentiles_rural_by_year,percentiles_income_by_year,percentiles_college_by_year,percentiles_unemployment_by_year,percentiles_not_english_proficient_by_year,percentiles_obesity_by_year,percentiles_diabetes_by_year,percentiles_smoking_by_year,percentiles_food_access_by_year,percentiles_exercise_access_by_year,percentiles_finance_by_year,percentiles_professional_by_year,percentiles_single_parent_households_by_year,percentiles_pop_density_by_year,percentiles_hu_density_by_year,percentiles_crime_by_year,percentiles_vehicle_crashes_by_year,percentiles_housing_problems_by_year
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.2814,12.9412,0.138000,0.217477,0.000259,0.11896,0.0,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.800000,52.224775,0.004963,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.10,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central,Medium Metro,0.360926,0.248048,0.114006,0.009449,0.086767,0.077400,0.174315,0.011257,0.141284,0.116676,0.126659,0.044246,0.031141,0.178215,0.775524,NaN,NaN,73.046680,89.250721,75.962840,0.006547,75,36,39,50,45,32,83,72,35,29,58,73,87,44,65,83,66,75,69,66,55,65,32
1,01001,2011,"Autauga County, Alabama",Autauga,Alabama,AL,0.315,0.2740,12.7858,0.149000,0.216246,0.000246,0.11400,0.0,0.873219,0.141000,0.130358,65.685521,0.777478,21859.0,36.772874,3.899220,0.365611,32.536382,75.400943,-86.644490,0.0945,0.695122,51622.0,0.000020,19.53,28.300000,0.000437,0.005000,0.514793,0.061602,0.024062,0.254795,0.002,0.010779,0.070508,0.448000,90.748703,53944.0,9967.40,78.853600,0.098860,307.8,0.170717,13.412318,42.572062,0.075488,0.028486,256.300000,2.499037,0.503662,3.186000,01,South,East South Central,Medium Metro,0.358613,0.252187,0.116547,0.012469,0.081611,0.069804,0.183148,0.009942,0.139650,0.112722,0.124984,0.052985,0.036290,0.179668,0.772153,NaN,NaN,73.105158,88.820510,75.690610,0.007137,79,36,40,53,4,32,80,71,45,33,67,48,86,43,77,80,66,68,69,66,57,64,31
2,01001,2012,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2460,12.7350,0.159811,0.217078,0.000210,0.12300,0.0,0.877878,0.159000,0.129745,63.094541,0.776964,22077.0,37.139559,4

In [117]:
df.to_csv('data/chr_census_dataset_tableau.csv')

In [88]:
df = pd.read_csv("data/chr_census_dataset_tableau.csv", index_col=0, dtype={"geo_code": str, "year": int, "state_fips": str})
df.head()

,geo_code,year,county_full_name,county_name,state_name,state_abbr,adult_obesity,adult_smoking,air_pollution_particulate_matter,children_in_poverty,college_completion,dentists,diabetes_prevalence,drinking_water_violations,driving_alone_to_work,excessive_drinking,frequent_mental_distress,gender_pay_gap,homeownership,housing_units,hu_density,income_inequality,insufficient_sleep,latitude,life_expectancy,longitude,low_birthweight,mammography_screening,median_household_income,mental_health_providers,mobility_index_std,motor_vehicle_crash_deaths,pcp,pct_aian,pct_female,pct_finance_insurance_and_real_estate,pct_hispanic,pct_male_pop_45_64,pct_moved_from_abroad,pct_not_proficient_in_english,pct_professional_management_and_scientific_services,pct_rural,pop_density,population,premature_death,preventable_hospital_stays,severe_housing_problems,sexually_transmitted_infections,single_parent_households,social_associations,teen_births,unemployment_rate,uninsured_children,violent_crime,food_access_score,exercise_access_score,poor_health,state_fips,region,division,cbsa_status,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_agriculture_hunting_and_mining,pct_arts_entertainment_and_accommodation_services,pct_construction,pct_educational_and_health_services,pct_information,pct_manufacturing,pct_public_administration,pct_retail_trade,pct_transportation_and_utilities,pct_wholesale_trade,pct_non_hispanic_black,pct_non_hispanic_white,life_exp_aapi,life_exp_aian,life_exp_black,life_exp_hispanic,life_exp_white,pct_aapi,percentiles_black_by_year,percentiles_white_by_year,percentiles_hispanic_by_year,percentiles_aapi_by_year,percentiles_aian_by_year,percentiles_rural_by_year,percentiles_income_by_year,percentiles_college_by_year,percentiles_unemployment_by_year,percentiles_not_english_proficient_by_year,percentiles_obesity_by_year,percentiles_diabetes_by_year,percentiles_smoking_by_year,percentiles_food_access_by_year,percentiles_exercise_access_by_year,percentiles_finance_by_year,percentiles_professional_by_year,percentiles_single_parent_households_by_year,percentiles_pop_density_by_year,percentiles_hu_density_by_year,percentiles_crime_by_year,percentiles_vehicle_crashes_by_year,percentiles_housing_problems_by_year
0,01001,2010,"Autauga County, Alabama",Autauga,Alabama,AL,0.300,0.2814,12.9412,0.138000,0.217477,0.000259,0.11896,0.0,0.868566,0.160546,0.128822,66.048439,0.775180,21530.0,36.219406,3.930128,0.367626,32.536382,75.607833,-86.644490,0.0883,0.668719,53255.0,0.000063,16.51,28.800000,52.224775,0.004963,0.515003,0.065322,0.023196,0.249884,0.002,0.009389,0.069964,0.422819,89.421390,53155.0,9778.10,91.803383,0.096612,370.0,0.179785,13.448252,47.513321,0.062076,0.028895,256.487012,2.499423,0.477935,3.292867,01,South,East South Central,Medium Metro,0.360926,0.248048,0.114006,0.009449,0.086767,0.077400,0.174315,0.011257,0.141284,0.116676,0.126659,0.044246,0.031141,0.178215,0.775524,NaN,NaN,73.046680,89.250721,75.962840,0.006547,75,36,39,50,45,32,83,72,35,29,58,73,87,44,65,83,66,75,69,66,55,65,32
1,01001,2011,"Autauga County, Alabama",Autauga,Alabama,AL,0.315,0.2740,12.7858,0.149000,0.216246,0.000246,0.11400,0.0,0.873219,0.141000,0.130358,65.685521,0.777478,21859.0,36.772874,3.899220,0.365611,32.536382,75.400943,-86.644490,0.0945,0.695122,51622.0,0.000020,19.53,28.300000,0.000437,0.005000,0.514793,0.061602,0.024062,0.254795,0.002,0.010779,0.070508,0.448000,90.748703,53944.0,9967.40,78.853600,0.098860,307.8,0.170717,13.412318,42.572062,0.075488,0.028486,256.300000,2.499037,0.503662,3.186000,01,South,East South Central,Medium Metro,0.358613,0.252187,0.116547,0.012469,0.081611,0.069804,0.183148,0.009942,0.139650,0.112722,0.124984,0.052985,0.036290,0.179668,0.772153,NaN,NaN,73.105158,88.820510,75.690610,0.007137,79,36,40,53,4,32,80,71,45,33,67,48,86,43,77,80,66,68,69,66,57,64,31
2,01001,2012,"Autauga County, Alabama",Autauga,Alabama,AL,0.341,0.2460,12.7350,0.159811,0.217078,0.000210,0.12300,0.0,0.877878,0.159000,0.129745,63.094541,0.776964,22077.0,37.139559,4

In [89]:
df.groupby("region")["life_expectancy"].mean()

region
Midwest      78.070585
Northeast    79.036178
South        75.847745
West         78.925922
Name: life_expectancy, dtype: float64

In [90]:
df.groupby("division")["life_expectancy"].mean()

division
East North Central    77.643638
East South Central    74.368324
Middle Atlantic       78.839183
Mountain              78.704828
New England           79.477213
Pacific               79.297942
South Atlantic        76.588062
West North Central    78.372489
West South Central    76.067326
Name: life_expectancy, dtype: float64

In [91]:
df.groupby("state_name")["life_expectancy"].mean()

state_name
Alabama                 74.402019
Alaska                  78.024712
Arizona                 78.043026
Arkansas                74.793519
California              79.808794
Colorado                80.010517
Connecticut             80.540941
Delaware                78.039430
District of Columbia    77.779764
Florida                 77.621533
Georgia                 75.844829
Hawaii                  81.441260
Idaho                   78.874688
Illinois                77.772692
Indiana                 76.820171
Iowa                    79.089488
Kansas                  77.648685
Kentucky                74.439125
Louisiana               74.886622
Maine                   78.378395
Maryland                78.412767
Massachusetts           80.152653
Michigan                77.918445
Minnesota               80.034864
Mississippi             73.799918
Missouri                76.544353
Montana                 78.162645
Nebraska                79.151094
Nevada                  77.809222
New

In [61]:
pd.DataFrame(top_negative_corr)

,life_expectancy
percentiles_unemployment_by_year,-0.436933
single_parent_households,-0.439138
low_birthweight,-0.497623
percentiles_single_parent_households_by_year,-0.526513
adult_obesity,-0.564590
percentiles_obesity_by_year,-0.564900
insufficient_sleep,-0.572529
adult_smoking,-0.580674
diabetes_prevalence,-0.595842
children_in_poverty,-0.599604
